In [1]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
import random
import numpy as np
import pickle

import os
import json
import pandas as pd
import copy
import random

import scipy
import scipy.optimize

import collections

In [2]:
data = []
with gzip.open('data\Grocery_and_Gourmet_Food_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

1143860
{'overall': 5.0, 'verified': True, 'reviewTime': '11 19, 2014', 'reviewerID': 'A1QVBUH9E1V6I8', 'asin': '4639725183', 'reviewerName': 'Jamshed Mathur', 'reviewText': 'No adverse comment.', 'summary': 'Five Stars', 'unixReviewTime': 1416355200}


In [3]:
required_vals=['overall','verified','reviewTime','reviewerID','asin','reviewText','summary','unixReviewTime']
data_cleaned=[]
cnt=0
imgcnt=0
for d in data: 
    add=True 
    for i in required_vals:
        if i not in d:
            add=False 
    if add:
        dcopy=copy.deepcopy(d) #not sure if this is really needed, will most likely remove
        if 'image' in dcopy:
            imgcnt+=1
            dcopy['hasImage']=1
        else:
            dcopy['hasImage']=0
        data_cleaned.append(dcopy)


    

In [4]:
print(imgcnt)
#this is  small so the image feature might be useless 

9474


In [5]:
#shuffle dataset
random.shuffle(data_cleaned)
train_data=data_cleaned[:int((len(data_cleaned)+1)*.80)]
val_data=data_cleaned[int((len(data_cleaned)+1)*.80):]

In [6]:
train_set=[]
test_set=[]
cnt=0

itemsPerUser=defaultdict(set)
usersPerItem=defaultdict(set)
items=set()
itemCount=defaultdict(int) # keep track of total times an item is reviewed

for review in train_data:
    user=review['reviewerID']
    item=review['asin']
    itemsPerUser[user].add(item)
    usersPerItem[item].add(user)
    items.add(item)
    itemCount[item]+=1

    flist=[]
    flist.append(user)
    flist.append(item)
    flist.append(review['overall'])
    train_set.append(flist)


for review in val_data:
    flist=[]
    flist.append(review['reviewerID'])
    flist.append(review['asin'])
    flist.append(review['overall'])
    test_set.append(flist)



# See "simple bias only latent factor model"
https://cseweb.ucsd.edu//classes/fa20/cse258-a/code/workbook4.html

This is a basic latent factor model from prev assignment (needs to be improved). Uses information only from user item pairs and ratings

In [7]:
labels= [int(d['overall']) for d in train_data]
ratingMean = sum([int(d['overall']) for d in train_data]) / len(train_data)
ratingMean

4.442140790742527

In [8]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

val_labels= [int(d['overall']) for d in val_data]
alwaysMean=[ratingMean]*len(val_labels)
baselineMSE=MSE(alwaysMean,val_labels)
baselineMSE

1.1367579747015757

In [9]:
occurrences = collections.Counter(labels)

In [10]:

N = len(train_set)
nUsers = len(itemsPerUser)
nItems = len(usersPerItem)
users = list(itemsPerUser.keys())
items = list(usersPerItem.keys())
alpha = ratingMean

userBiases = defaultdict(float)
itemBiases = defaultdict(float)
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))
    
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in train_set]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(train_set)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in train_set:
        u,i = d[0], d[1]
        pred = prediction(u, i)
        diff = pred - int(d[2])
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return np.array(dtheta)


In [11]:

#basic  latent factor model from prev assignment (needs to be improved)
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, .0001))

MSE = 1.1308026933786226
MSE = 1.123494643180019
MSE = 1.3801577839909287
MSE = 1.1219526597275666
MSE = 1.1144653694943045
MSE = 1.1137677095780343
MSE = 1.1110799659060562
MSE = 1.1019753271072044
MSE = 1.0853022823796694
MSE = 1.0712746448360957
MSE = 1.0464222575040165
MSE = 1.0342632561866645
MSE = 1.0311546960455593
MSE = 1.031264525691905
MSE = 1.0288459044939853
MSE = 1.0261955393259998
MSE = 1.0232785110486933
MSE = 1.024613985531601
MSE = 1.0233062589108715
MSE = 1.024146519274313
MSE = 1.0232865330801724
MSE = 1.0231201132102732
MSE = 1.0231330902609403
MSE = 1.0232722170502955
MSE = 1.0230834381404676
MSE = 1.0231411982457022
MSE = 1.0229669499701335
MSE = 1.0228033787692967
MSE = 1.0226462128638882
MSE = 1.0225521725230073
MSE = 1.0225780320036422
MSE = 1.022527852724066
MSE = 1.0225537806828744
MSE = 1.022587249789403
MSE = 1.0226776380940696
MSE = 1.022631136113244
MSE = 1.0226527677596342
MSE = 1.0226357415238818
MSE = 1.0226138061896677
MSE = 1.0225708108061429
MSE = 1

(array([4.41754651e+00, 1.38768021e-02, 7.52129313e-03, ...,
        1.67582618e-03, 6.30508625e-03, 5.96515691e-03]),
 1.0654853592995777,
 {'grad': array([-6.46130275e-06,  1.10523183e-09,  8.17370982e-11, ...,
          2.75804715e-10, -4.44652679e-09, -1.00949612e-09]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 59,
  'nit': 50,
  'warnflag': 0})

In [23]:
X=[]
y=[]
cnt=0
userSet=set(users)# doing this to improve search speed
itemSet=set(items)
for  u,i,s in test_set:
    # if cnt%1000==0:
    #     print(cnt)
    # cnt+=1
    y.append(s)
    case_1=u in userSet
    case_2=i in itemSet
    if case_1 and case_2:
        X.append(prediction(u,i))
        continue
    elif case_1 and not case_2:
        p=alpha+userBiases[u]
        if p<0:
            p=0
        if p>5:
            p=5
        X.append(p)
        continue
    elif case_2 and not case_1:
        p=alpha+itemBiases[i]
        if p<0:
            p=0
        if p>5:
            p=5
        X.append(p)
        continue
    else:
        X.append(alpha)
    

In [24]:
MSE_val=MSE(X,y)
MSE_val

1.061770693765833